In [77]:
# Import necessary libraries
import base64
import json

import os
import fitz
import requests

## Save page 3 with clean contents

In [78]:
src = fitz.open("../PyMuPDF_playground/576_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(1)
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page = new_doc.new_page(-1, width=r2.width, height=r2.height)
new_page.show_pdf_page(
    new_page.rect,
    src,
    page.number,
    clip=r2,
)

img_list = new_page.get_images(full=True)

for i, img in enumerate(img_list):
    rect = new_page.get_image_bbox(img)
    if rect[0] > 0 and rect[1] > 0 and rect[2] > 0 and rect[3] > 0:
        wanted_xref = img[0]
        print(rect, i, img)

Rect(27.194091796875, 61.0030517578125, 601.9796142578125, 444.04248046875) 5 (92, 0, 799, 532, 8, 'ICCBased', '', 'Im5', 'DCTDecode', 94)


After finding out where the single image is, let's extract it.

In [79]:
if not wanted_xref is None:
    image = new_doc.extract_image(wanted_xref)
    # write extracted image to file
    image_title = src.name.split('.')[2].split('/')[2]+"_image_"+str(wanted_xref)
    image_path = os.path.join(
        '/home/funny/Documents/Projects/230701-Parsing-Tool/augustin-plugin/sample_data/',
        image_title+"."+image['ext']
        )
    with open(
        (image_path),
        "wb",
    ) as imgout:
        imgout.write(image["image"])
    
else:
    print("No image found")

## Extract text

In [80]:
r = new_page.rect
r.y0 = (r.y1 * 2) / 3  # two third of page height
r

Rect(0.0, 566.9293212890625, 637.7949829101562, 850.3939819335938)

In [81]:
new_page.get_textbox(r)

'I\nch verkaufe den Augustin im fünften  Bezirk \nvor einem Hofer. Den Verkaufsplatz  finde \nich okay, denn die Menschen dort sind sehr \nfreundlich und es gibt keine Aggressivität. \nIch habe drei Jahre auf einen Augustin-Aus-\nweis gewartet (es gibt eine Warteliste, Anm.). \nDavor habe ich die Straßenzeitung MO verkauft, \naber mit dem Augustin läuft es  besser. Ich bin \n2017 mit meiner Frau und meinen beiden Kin-\ndern hergekommen. Die Eltern meiner Frau \nsind schon seit 2000 in Wien, also haben sie \nmir  dabei geholfen, hier Fuß zu fassen.\nIch bin in Wien, weil ich hier Geld verdienen \nkann. In Rumänien gibt es fast keine Arbeit. Dort \nhabe ich im Holzlager meiner Eltern gearbeitet, \naber man verdient mit dem Job fast gar nichts. In \nBezug auf Jobs habe ich keine hohen Ansprüche. \nIch arbeite das, was man von mir verlangt. Nur \nist es schwer für mich zu kommunizieren, weil \nich die Sprache nicht verstehe. Ich habe auch \nbei Uber gearbeitet und bei einem Gemüsehänd-\nle

In [94]:
r2 = new_page.rect
r2.y0 = r2.y1 / 2  # half of page height
r2.y1 = r2.y1 * 2 / 3  - 3# two third of page height -3 so it is not overlapping
r2

Rect(0.0, 425.1969909667969, 637.7949829101562, 563.9293212890625)

In [96]:
string = new_page.get_textbox(r2)
array = string.splitlines()
array

['Cosmin',
 'Eine bessere Zukunft',
 'PRotoKoLL: syLviA gALosi',
 'ÜBeRsetZUng: RoXAnA tiPLeA',
 'Foto: MARio LAng']